In [1]:
import glob, os
import random
import shutil
import json
import pandas as pd

In [2]:
# Util functions
def tag_loc(tag):
    return (tag + 1) if tag != 14 else 8

def classify(score):
    if score < 2:
        label_index = 1
    elif score >= 2 and score < 3:
        label_index = 2
    elif score >= 3 and score < 4:
        label_index = 3
    elif score >= 4:
        label_index = 4
   
    return label_index


## Split data into training and test set

In [107]:
# Shuffle dataset
datadir = '/home/wei-chan.hsu/Dokumente/Thesis/src/annotation/data/all/data_json/'
all_files = os.listdir(os.path.abspath(datadir))
data_files = list(filter(lambda file: file.endswith('.json'), all_files))

print("Before shuffle: ", data_files[:5])
random.shuffle(data_files)
print("After shuffle: ", data_files[:5])

Before shuffle:  ['cow239-3.json', 'cow1-14.json', 'cow1407-6.json', 'cow1407-1.json', 'cow161-0.json']
After shuffle:  ['cow234-2.json', 'cow24-3.json', 'cow80-14.json', 'cow254-2.json', 'cow56-4.json']


In [108]:
# SPlit data into two sets
split_ratio = 0.7
split = int(split_ratio * len(data_files))
training_set = data_files[:split]
test_set = data_files[split:]

print("Training set ({} samples): ".format(len(training_set)), training_set[:5])
print("Test set ({} samples): ".format(len(test_set)), test_set[:5])

Training set (350 samples):  ['cow234-2.json', 'cow24-3.json', 'cow80-14.json', 'cow254-2.json', 'cow56-4.json']
Test set (151 samples):  ['cow1403-4.json', 'cow190-4.json', 'cow264-1.json', 'cow204-7.json', 'cow161-6.json']


In [123]:
def copy_files(src, dst):
    for file in src:
        shutil.copy(file, dst)

In [122]:
dst_train = '/home/wei-chan.hsu/Dokumente/Thesis/src/input_st-gcn/cows_train/'
dst_train

'/home/wei-chan.hsu/Dokumente/Thesis/src/input_st-gcn/cows_train/'

In [127]:
# Copy JSON files into directories 
src_train = [(datadir + file) for file in training_set]
src_test = [(datadir + file) for file in test_set]
dst_train = '/home/wei-chan.hsu/Dokumente/Thesis/src/input_st-gcn/lameness/cows_train/'
dst_test = '/home/wei-chan.hsu/Dokumente/Thesis/src/input_st-gcn/lameness/cows_test/'

copy_files(src_train, dst_train)
copy_files(src_test, dst_test)

print(glob.glob(dst_train[:5]))

['/home']


In [129]:
# Generate trainig and test label files
label_table = pd.read_csv('/home/wei-chan.hsu/Dokumente/Thesis/src/annotation/data_labels.csv')

def files2labels(dataset, vname):
    data_dict = {}
    for file in dataset:
        files.append(os.path.splitext(f)[0])
        filename = file.split('.')[0]
        cow_id = filename.split('-')[0]
        tag = filename.split('-')[1]
        label = label_table[(label_table['cow_id'] == cow_id)].iloc[0, tag_loc(int(tag))]
        item = {
            "has_skeleton": True, 
            "label": label,
            "label_index": classify(label)
        }
        data_dict[filename] = item
        #with open("/home/wei-chan.hsu/Dokumente/Thesis/src/input_st-gcn/{}.txt".format(vname), "a") as f:
            #f.write("text\n")
    with open("/home/wei-chan.hsu/Dokumente/Thesis/src/input_st-gcn/lameness/{}.json".format(vname), "w") as out_file:
        json.dump(data_dict, out_file, indent=4)   

files2labels(training_set, 'train_label')  
files2labels(test_set, 'test_label')  